In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from colorcloud.cheng2023TransRVNet import TransVRNet, SemanticSegmentationTask
from colorcloud.cheng2023TransRVNet import TransRVNet_loss
from colorcloud.cheng2023TransRVNet import RandomRotationTransform, RandomDroppingPointsTransform, RandomSingInvertingTransform
import lightning as L
import numpy as np
import wandb
from lightning.pytorch.loggers import WandbLogger
from datetime import datetime
import torch
import torchvision
import lightning as L
from colorcloud.behley2019iccv import SemanticKITTIDataset, SphericalProjection, ProjectionTransform, ProjectionToTensorTransform, SemanticSegmentationLDM
from torchvision.transforms import v2
from torch.utils.data import DataLoader

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Setup

In [3]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


### Convolutions' Parameters

In [4]:
mrciam_p = {
    "p1": {
        "b1_in": 1,
        "b1_out1": 32,
        "b1_out2": 32,
        "b1_out3": 32,
        "b2_in": 64,
        "b2_out": 64,
        "b3_in": 64,
        "b3_out1": 64,
        "b3_out2": 64,
        "b3_out3": 64,
        "output": 64,
    },
    "p2": {
        "b1_in": 3,
        "b1_out1": 32,
        "b1_out2": 32,
        "b1_out3": 32,
        "b2_in": 64,
        "b2_out": 64,
        "b3_in": 64,
        "b3_out1": 64,
        "b3_out2": 64,
        "b3_out3": 64,
        "output": 64,
    },
    "output_conv": 192
}
encoder_p = {
    "module_1": {
        "in_channels": 192,
        "conv2_in_channels": 128,
        "conv2_out_channels": 256,
        "dilated_conv_out_channels": 256,
        "residual_out_channels": 256
    },
    "module_2": {
        "in_channels": 256,
        "conv2_in_channels": 256,
        "conv2_out_channels": 256,
        "dilated_conv_out_channels": 256,
        "residual_out_channels": 256
    }
}

decoder_p = {
    "in_channels": 264,
    "conv2_in_channels": 128,
    "dilated_conv_in_channels": 128,
    "dilated_conv_out_channels": 64,
    "output": 32
}

p_bntm = {
    "window_size": (4,4),
    "embed_dim": int(encoder_p["module_2"]["residual_out_channels"]/8)
}

## Lightning Training

In [5]:
aug_tfms = v2.Compose([
    RandomDroppingPointsTransform(),
    RandomRotationTransform(),
    RandomSingInvertingTransform(),
])

In [6]:
model = TransVRNet(mrciam_p, encoder_p, decoder_p, p_bntm).to(device)
loss_fn = TransRVNet_loss(device) 

In [7]:
data = SemanticSegmentationLDM(proj_kargs={'fov_up_deg':4., 'fov_down_deg':-26., 'W':1024, 'H':64}, proj_style='spherical', 
                               train_batch_size=1, eval_batch_size=1, num_workers=2, aug_tfms=aug_tfms)
data.setup('fit')
epoch_steps = len(data.train_dataloader())

In [8]:
n_epochs = 10
learner = SemanticSegmentationTask(
    model,
    loss_fn,
    data.viz_tfm, 
    total_steps=n_epochs*epoch_steps
)

In [9]:
model_name = "TransVRNet"
timestamp = datetime.now().strftime("%d/%m/%Y_%H:%M:%S")
experiment_name = f"{model_name}_{timestamp}"

wandb_logger = WandbLogger(project="colorcloud", name=experiment_name, log_model="all")
wandb_logger.watch(learner.model, log="all")

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: matheus-lla (matheus-ufg). Use `wandb login --relogin` to force relogin


wandb: logging graph, to disable use `wandb.watch(log_graph=False)`


In [10]:
# train model
trainer = L.Trainer(max_epochs=n_epochs, logger=wandb_logger)
trainer.fit(learner, data)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 3050 Laptop GPU') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name           | Type                   | Params | Mode 
------------------------------------------------------------------
0 | model          | TransVRNet             | 10.1 M | train
1 | loss_fn        | TransRVNet_loss        | 0      | train
2 | viz_tfm        | ProjectionVizTransform | 0      | train
3 | dropout        | Dropout                | 0      | train
4 | train_accuracy | MulticlassAccuracy     | 0      | train
5 | val_accuracy   | MulticlassAc

/opt/conda/lib/python3.10/site-packages/lightning/pytorch/loops/fit_loop.py:298: The number of training batches (5) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Epoch 0:   0%|          | 0/5 [00:00<?, ?it/s] 

/opt/conda/lib/python3.10/site-packages/torch/autograd/graph.py:744: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at /opt/conda/conda-bld/pytorch_1712608935911/work/aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass
/workspace/colorcloud/colorcloud/cheng2023TransRVNet.py:1166: UserWarning: std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at /opt/conda/conda-bld/pytorch_1712608935911/work/aten/src/ATen/native/ReduceOps.cpp:1807.)
  self.logger.experiment.log({"ud/" + str(n): ((optim_step.std())/(p_old[n].std() + 1e-5)).log10()}, step=self.step_idx)


Epoch 9: 100%|██████████| 5/5 [02:19<00:00,  0.04it/s, v_num=1yaz]

wandb: WARNING Tried to log to step 5 that is less than the current step 7. Steps must be monotonically increasing, so this data will be ignored. See https://wandb.me/define-metric to log data out of order.
wandb: WARNING Tried to log to step 5 that is less than the current step 7. Steps must be monotonically increasing, so this data will be ignored. See https://wandb.me/define-metric to log data out of order.
wandb: WARNING Tried to log to step 5 that is less than the current step 7. Steps must be monotonically increasing, so this data will be ignored. See https://wandb.me/define-metric to log data out of order.
wandb: WARNING Tried to log to step 5 that is less than the current step 7. Steps must be monotonically increasing, so this data will be ignored. See https://wandb.me/define-metric to log data out of order.
wandb: WARNING Tried to log to step 5 that is less than the current step 7. Steps must be monotonically increasing, so this data will be ignored. See https://wandb.me/defin

`Trainer.fit` stopped: `max_epochs=10` reached.


Epoch 9: 100%|██████████| 5/5 [02:20<00:00,  0.04it/s, v_num=1yaz]
